In [1]:
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn')

from scipy.stats import linregress

In [2]:
df = pd.read_parquet('df.parquet.gzip')

In [3]:
df.dropna(inplace=True)

In [4]:
df.index = df["date"]
df.drop(["date"], axis=1, inplace=True)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score

def get_feature_matrix(start_ind, end_ind):
    return df[start_ind:end_ind].drop(columns=['date','open','high','low','close','wclose','delta'])
def get_training_labels(start_ind, end_ind):
    return df[start_ind:end_ind]['delta']

In [6]:
X = df.drop(["delta"],axis=1)
y = df["delta"]
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)

In [7]:
GS1 =  GridSearchCV(DecisionTreeClassifier(random_state = 42), 
                    {'max_depth': [4, 5, 6, 7, 8, 9, 10,11], 'min_samples_leaf': [2,3, 4, 5, 6, 7]},
                    cv=5)

GS1.fit(X_train, y_train)
y_pred = GS1.predict(X_test)
GS1.score(X_test, y_test)

0.7981293842556508

In [13]:
dt = GS1.best_estimator_
importances = dict(zip(df.drop(["delta"],axis=1).columns, dt.feature_importances_))

k = [i for i in importances.keys() if importances[i] !=0]

In [14]:
k

['so_3', 'so_8', 'cci_15', 'slope_3', 'slope_4', 'slope_30']

In [15]:
df0 = df.drop(k, axis=1)

X = df.drop(["delta"],axis=1)
y = df["delta"]
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)

In [20]:
GS2 =  GridSearchCV(KNeighborsClassifier(), 
                    {'leaf_size': [6, 8, 10, 20, 40, 100], 'weights': ['uniform','distance']},
                    cv=5)

GS2.fit(X_train, y_train)
y_pred = GS2.predict(X_test)
GS2.score(X_test, y_test)

0.7466874512860483

In [19]:
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}